# Каких пользователей и сколько закачаем?

TLDR:

Из-за ограничений API сэмплируем данные: берём по 1000 пользователй каждой группы

Делаем небольшой offset для качественного сэмплирования

Закачиваем характеристики пользователей (пол и анкету "Любимая музыка", ничего лишнего)

Сохраняем всё в файле big_data.csv

In [4]:
 # coding: utf-8
# наше всё
import numpy as np
import pandas as pd

#настройки pandas, с которыми лучше почти всегда
pd.set_option('display.max_rows', 45000)
pd.set_option('display.max_columns', 50000)
pd.set_option('display.max_colwidth', 5000)

import requests
import time

In [2]:
access_token='не_забудь_вставить_сюда_свежий_токен'

In [5]:
# чтобы узнать, как получился этот файл, см. файл bands_findings.ipynb
group_df=pd.read_csv('group_df.csv')

In [6]:
group_df.sample(2)

,id,is_closed,name,photo_100,photo_200,photo_50,screen_name,type
1,59992118,0,Группа «Пикник»,https://sun1-20.userapi.com/c856132/v856132130/c688f/OLCwKy3GnL4.jpg?ava=1,https://sun1-93.userapi.com/c856132/v856132130/c688e/9FltsObk29Q.jpg?ava=1,https://sun1-98.userapi.com/c856132/v856132130/c6890/FoKxnfYG0mM.jpg?ava=1,piknik_info,page
18,144798458,0,Стас Михайлов,https://sun1-21.userapi.com/c856124/v856124700/1ca48c/7ICJqR39oIA.jpg?ava=1,https://sun1-98.userapi.com/c856124/v856124700/1ca48b/FMqEbDE5dO4.jpg?ava=1,https://sun1-90.userapi.com/c856124/v856124700/1ca48d/w5TxCvLZb6A.jpg?ava=1,mihailov.group,group


In [5]:
# функция делает запрос к API vk.com
# выдаёт список пользователей данной группы (порциями по 1000 штук)
# больше за раз нельзя из-за ограничения API
# на вход vk_id сообщества, отступ в головах и параметр для отладки
def get_group_part(id, offset, verbose):
    url='https://api.vk.com/method/'
    method='groups.getMembers'
    params = (
        ('group_id', id),
        ('v', '5.52'),
        ('offset', offset),
        ('access_token', access_token),
    )
    
    response = requests.get(url+method, params=params)
    
    # если мы включим отладочный параметр, то увидим какую ошибку выдаёт API
    if verbose==1:
        print(response.text[0:3000])
    
    #сразу выкинем мусор из ответа
    return response.json()['response']['items']


# полный список порциями
# в этой реализации не нужен
# на входе только id сообщества
def get_full_user_list(id):
    # идём порциям по 1000 человек (максимум, что даёт API)
    rng=range(0,1000*1000,1000)
    
    full_lst=[]
    for ofs in rng:
        print(ofs)
        try:
            lst=get_group_part(id, ofs, 0)
        except:
            lst=[]
        time.sleep(1)
        full_lst=full_lst+lst
        
    return full_lst

# функция делает запрос к API vk.com
# на выход получает строчку с праметрами сообщества
# выдёт датафрейм с id пользователей, состоящих в сообществе
def get_group_users(df):
    lst=[]
    
    user_df=pd.DataFrame()
    for group in df.values:
        # т.к. процесс долгий, не помешает выводить название группы перед началом
        print(group[2])
        # если упадём по ошибке, запустим ещё раз и узнаем, на что ругается API
        # отсутп 500 - достаточный для рандома для простых смертных
        try:
            lst=get_group_part(group[0], 500, 0)
        except:
            get_group_part(group[0], 500, 1)
            lst=[]
        # здесь задержка в секунду решает все наши проблемы
        time.sleep(1)
        # и сразу сделаем нормальный датафрейм
        user_df_buf=pd.DataFrame(lst, columns=['user_id'])
        user_df_buf['group']=group[2]
        user_df_buf['group_id']=group[0]
        
        user_df=pd.concat([user_df, user_df_buf])
        
    return user_df

In [6]:
# функция делает запрос к API vk.com
# на вход получает id пользователя
# на выходе датафрейм, в которой есть id польователя, 
# имя, фамилия, 
# его пол (1 - женский, 2- мужской, 0 - всё сложно)
# а также то, что пользователь оставил в своей анкете "любимая музыка"
def user_info(id):
    url='https://api.vk.com/method/'
    method='users.get'
    params = (
        ('user_id', id),
        # sex, не gender, но это именно м-ж
        ('fields', 'sex,music'),
        ('v', '5.52'),
        ('access_token', access_token),
    )

    response = requests.get(url+method, params=params)
    try:
        data=pd.io.json.json_normalize(response.json()['response'])
    except:
        print(response.text[0:3000])
        data=pd.DataFrame()
        
    return data

# функция для обработки исключений в анкете
def music_parse(df):
    # может быть так, что поля в ответе просто нет, тогда создадим
    if 'music' not in df.columns:
        df['music']='No info'
            
    # для того, чтобы проще было хранить в csv, заменим ; от эстетов на , 
    df['music']=df['music'].str.replace(';', ',')
    #  а бывает так, что поле есть, но пустое, вставим заглушку, чтобы дальше было проще жить
    df['music']=np.where(df['music']=='','No info', df['music'])
    
    df=df[['first_name', 'id', 'last_name', 'music', 'sex']]
    return df

# основной цикл
# идём по списку пользовтелей и сразу пишем в файл
def get_full_info(df):
    i=0
    for user in df['user_id'].values:
        # получаем инфу о человеке
        tmp_df=user_info(user)
        # обрабатываем её
        tmp_df=music_parse(tmp_df)
        
        # записываем результат
        tmp_df.to_csv('big_data.csv', sep=';', index=False, mode='a+', header=False)
        
        # эмпирически выведеная задержка, которая делает всех счастливыми
        if i%2==0:
            time.sleep(0.6)
        # самодельный прогресс-бар
        if i%100==0:
            print(
                str(
                    round(100*i/len(df),0)
                   )+'%'
                 )
        i=i+1 
            

In [11]:
# перемешал для того, чтобы по данным можно было делать выводы даже если что-то упадёт
# при сэмплировании не так важно
from sklearn.utils import shuffle
user_data = shuffle(user_data)

In [12]:
user_data.sample(4)

,user_id,group,group_id
780,158948430,Стас Михайлов,144798458
971,339521,Армия АлисА,383
948,12015313,FLЁUR / ФЛЁР,12093489
576,236753,Армия АлисА,383


In [13]:
# сохраняем первый файл
user_data.to_csv('users.csv', index=False)

In [14]:
%%time
# самое тяжёлое и страшное запускается здесь
get_full_info(user_data)

0.0%
3.0%
7.0%
10.0%
13.0%
17.0%
20.0%
23.0%
27.0%
30.0%
33.0%
37.0%
40.0%
43.0%
47.0%
50.0%
53.0%
57.0%
60.0%
63.0%
67.0%
70.0%
73.0%
77.0%
80.0%
83.0%
87.0%
90.0%
93.0%
97.0%
CPU times: user 1min 35s, sys: 3.07 s, total: 1min 38s
Wall time: 19min 48s


In [15]:
# загрузим полученное, посмотрим ему в глаза
user_full_data=pd.read_csv('big_data.csv', header=None, sep=';')
user_full_data.columns=['first_name', 'id', 'last_name', 'music', 'sex']

In [16]:
user_full_data.sample(20)

,first_name,id,last_name,music,sex
37371,Андрей,738555,Долгополов,"Metallica, The Offspring, Nirvana, RHCP, Papa Roach, Limp Bizkit, Cold, Staind, Drowning pool, Sum 41, Blink 182, Наив, Тараканы, linkin park, kiss, hoobastank, Lumen, P.O.D",2
29413,Анастасия,1171112,Малофеева,rus rock,1
31790,Екатерина,130349,Хмелевская,No info,1
24388,Алёна,708341,Лещинская,No info,1
24466,Любовь,998052,Ли,No info,1
40801,Андрей,182367,Осипов,No info,2
29941,Ольга,334497,Каплан,the Doors,1
32841,Мария,70412444,Апкаликова,No info,1
25804,Павел,323386,Данилов,No info,2
26222,Алеся,942228,Шляпникова,No info,1
